İŞ PROBLEMİ:

Bir tatil sitesi yaptığı satışların bazı özelliklerini kullanarak seviye tabanlı (level based) yeni satış tanımları oluşturmak ve bu yeni satış tanımlarına göre segmentler oluşturup bu segmentlere göre yeni gelebilecek müşterilerin şirkete ortalama ne kadar kazandırabileceğini tahmin etmek istemektedir.Örneğin Antalya’dan Herşey Dahil bir otele yoğun bir dönemde gitmek isteyen bir müşterinin ortalama ne kadar kazandırabileceği belirlenmek isteniyor.

Veri Seti Hikayesi: 
Veri seti bir tatil sitesinin yaptığı satışların fiyatlarını ve bu satışlara ait bilgiler içermektedir. Veri seti her satış işleminde oluşan kayıtlardan meydana gelmektedir. Bunun anlamı tablo tekilleştirilmemiştir. Diğer bir ifade ile müşteri birden fazla alışveriş yapmış olabilir.

Değişkenler: 
SaleId : Satış id SaleDate : Satış Tarihi CheckInDate : Müşterinin otele girişi tarihi Price : Satış için ödenen fiyat ConceptName: Otel konsept bilgisi SaleCityName: Otelin bulunduğu şehir bilgisi CInDay:Müşterinin otele giriş günü SaleCheckInDayDiff: Check in ile giriş tarihi gün farkı Season:Otele giriş tarihindeki sezon bilgisi

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", 500)

In [2]:
df = pd.read_csv("kural_tabanl_dataset.csv")
df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons
0,415122,2022-12-03,2022-12-03,79.304029,Herşey Dahil,Antalya,Saturday,0,Low
1,415103,2022-12-03,2022-12-03,45.970696,Yarım Pansiyon,Antalya,Saturday,0,Low
2,404034,2022-09-12,2022-09-13,77.838828,Herşey Dahil,Antalya,Tuesday,1,High
3,415094,2022-12-03,2022-12-10,222.710623,Yarım Pansiyon,İzmir,Saturday,7,Low
4,414951,2022-12-01,2022-12-03,140.476190,Yarım Pansiyon,İzmir,Saturday,2,Low


1.VERİYE GENEL BAKIŞ

In [3]:
print("##################### Shape #####################")
print(df.shape)
print("##################### Head #####################")
print(df.head())
print("##################### NA #####################")
print(df.isnull().sum())
print("##################### INFO #####################")
print(df.info())

##################### Shape #####################
(59164, 9)
##################### Head #####################
   SaleId    SaleDate CheckInDate       Price     ConceptName SaleCityName    CInDay  SaleCheckInDayDiff Seasons
0  415122  2022-12-03  2022-12-03   79.304029    Herşey Dahil      Antalya  Saturday                   0     Low
1  415103  2022-12-03  2022-12-03   45.970696  Yarım Pansiyon      Antalya  Saturday                   0     Low
2  404034  2022-09-12  2022-09-13   77.838828    Herşey Dahil      Antalya   Tuesday                   1    High
3  415094  2022-12-03  2022-12-10  222.710623  Yarım Pansiyon        İzmir  Saturday                   7     Low
4  414951  2022-12-01  2022-12-03  140.476190  Yarım Pansiyon        İzmir  Saturday                   2     Low
##################### NA #####################
SaleId                 0
SaleDate               0
CheckInDate            0
Price                 13
ConceptName            0
SaleCityName           0
CInDay         

In [4]:
# Soru 1: Kaç unique şehir vardır?
df["SaleCityName"].nunique()

6

In [5]:
# Soru 2: Kaç unique Concept vardır?
df["ConceptName"].unique()

array(['Herşey Dahil', 'Yarım Pansiyon', 'Oda + Kahvaltı'], dtype=object)

In [6]:
# Soru 3: Hangi Concept’den kaçar tane satış gerçekleşmiş?
df["ConceptName"].value_counts()

ConceptName
Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: count, dtype: int64

In [7]:
# Soru 4: Şehirlere göre satışlardan toplam ne kadar kazanılmış?
df.groupby("SaleCityName").agg({"Price": "sum"}).sort_values("Price", ascending=False)

,Price
SaleCityName,
Antalya,2.041911e+06
Muğla,6.658422e+05
Aydın,5.732960e+05
İzmir,1.659348e+05
Diğer,1.545723e+05
Girne,2.706503e+04


In [8]:
# Soru 5: Concept türlerine göre göre ne kadar kazanılmış?
df.groupby("ConceptName").agg({"Price": "sum"}).sort_values("Price", ascending=False)

,Price
ConceptName,
Herşey Dahil,3.332911e+06
Yarım Pansiyon,1.744024e+05
Oda + Kahvaltı,1.213084e+05


In [9]:
# Soru 6: Şehirlere göre PRICE ortalamaları nedir?
df.groupby("SaleCityName").agg({"Price": "mean"}).sort_values("Price", ascending=False)

,Price
SaleCityName,
İzmir,66.267905
Antalya,64.521474
Muğla,62.461746
Girne,59.483579
Aydın,53.855896
Diğer,47.707497


In [10]:
# Soru 7: Conceptlere göre PRICE ortalamalarınedir?
df.groupby("ConceptName").agg({"Price": "mean"}).sort_values("Price", ascending=False)

,Price
ConceptName,
Herşey Dahil,62.672260
Oda + Kahvaltı,50.252010
Yarım Pansiyon,49.030742


In [11]:
# Soru 8: Şehir-Concept kırılımında PRICE ortalamaları nedir?
df.groupby(["SaleCityName", "ConceptName"]).agg({"Price": "mean"})

Price
SaleCityName ConceptName              
Antalya      Herşey Dahil    64.519028
             Oda + Kahvaltı  63.504816
             Yarım Pansiyon  67.190613
Aydın        Herşey Dahil    53.995467
             Oda + Kahvaltı  34.458787
             Yarım Pansiyon  30.016543
Diğer        Herşey Dahil    84.771093
             Oda + Kahvaltı  37.599082
             Yarım Pansiyon  42.113440
Girne        Herşey Dahil    97.680970
             Oda + Kahvaltı  39.775536
             Yarım Pansiyon  53.247838
Muğla        Herşey Dahil    63.020053
             Oda + Kahvaltı  59.037763
             Yarım Pansiyon  45.120924
İzmir        Herşey Dahil    74.701519
             Oda + Kahvaltı  41.320342
             Yarım Pansiyon  59.610053

2.VERİ MANİPÜLASYONU

In [12]:
# SaleCheckInDayDiff değişkenini kategorik bir değişkene çeviriniz.
# Aralıklar için "Last Minuters", "Potential Planners", "Planners", "Early Bookers“ isimlerini kullanabilirsiniz.
labels = ["Last Minuters", "Potential Planners", "Planners", "Early Bookers"]
df["Sale_Check_Cat"] = pd.cut(df["SaleCheckInDayDiff"], bins=[0, 7, 30, 90, float("inf")], labels=labels, right=False)
df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons,Sale_Check_Cat
0,415122,2022-12-03,2022-12-03,79.304029,Herşey Dahil,Antalya,Saturday,0,Low,Last Minuters
1,415103,2022-12-03,2022-12-03,45.970696,Yarım Pansiyon,Antalya,Saturday,0,Low,Last Minuters
2,404034,2022-09-12,2022-09-13,77.838828,Herşey Dahil,Antalya,Tuesday,1,High,Last Minuters
3,415094,2022-12-03,2022-12-10,222.710623,Yarım Pansiyon,İzmir,Saturday,7,Low,Potential Planners
4,414951,2022-12-01,2022-12-03,140.476190,Yarım Pansiyon,İzmir,Saturday,2,Low,Last Minuters


In [13]:
# SaleCityName, ConceptName, Seasons kırılımında ortalama ödenen ücret ve yapılan işlem sayısı cinsinden inceleyiniz ?
df.groupby(["SaleCityName", "ConceptName", "Seasons"]).agg({"Price": ["mean", "count"]})

Price       
                                           mean  count
SaleCityName ConceptName    Seasons                   
Antalya      Herşey Dahil   High      64.920065  27126
                            Low       61.552427   3667
             Oda + Kahvaltı High      66.267969    303
                            Low       60.666730    295
             Yarım Pansiyon High      73.259029    118
                            Low       62.001678    138
Aydın        Herşey Dahil   High      54.946560  10103
                            Low       33.680699    473
             Oda + Kahvaltı High      30.387530     27
                            Low       44.451874     11
             Yarım Pansiyon High      32.626152     20
                            Low       25.271799     11
Diğer        Herşey Dahil   High      83.787273    364
                            Low       87.310882    141
             Oda + Kahvaltı High      34.839543    349
                            Low       39.959569    408
             Yarım Pansiyon High      39.731916    516
                            Low       42.953978   1462
Girne        Herşey Dahil   High     103.935448     55
                            Low       90.935944     51
             Oda + Kahvaltı High      39.479630     66
                            Low       40.043067     73
             Yarım Pansiyon High      58.383668    100
                            Low       48.578903    110
Muğla        Herşey Dahil   High      63.384036   9582
                            Low       48.304086    237
             Oda + Kahvaltı High      61.264691    592
                            Low       37.774192     62
             Yarım Pansiyon High      45.823698    177
                            Low       32.681812     10
İzmir        Herşey Dahil   High      74.748363   1234
                            Low       74.308287    147
             Oda + Kahvaltı High      54.141797     86
                            Low       33.555235    142
             Yarım Pansiyon High      87.657303    118
                            Low       55.350626    777

In [14]:
# SaleCityName, ConceptName, Seasons kırılımının çıktısını PRICE'a göre sıralayınız.
# Elde ettiğiniz çıktıyı agg_df olarak kaydediniz.
# Indekste yer alan isimleri değişken ismine çeviriniz.
agg_df = df.groupby(["SaleCityName", "ConceptName", "Seasons"]).agg({"Price": "mean"}).sort_values("Price", ascending=False)
agg_df = agg_df.reset_index()
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price
0,Girne,Herşey Dahil,High,103.935448
1,Girne,Herşey Dahil,Low,90.935944
2,İzmir,Yarım Pansiyon,High,87.657303
3,Diğer,Herşey Dahil,Low,87.310882
4,Diğer,Herşey Dahil,High,83.787273


In [15]:
# Yeni seviye tabanlı satışları tanımlayınız ve veri setine değişken olarak ekleyiniz.
# Yeni oluşturacağımız Sales_Level_Based değişkeni sırası ile; SaleCityName, ConceptName, Seasons verilerinin her birinin 
# birleşimini gösterecek şekilde yeni değişken oluşturunuz.
agg_df["Sales_Level_Based"] = agg_df[["SaleCityName", "ConceptName", "Seasons"]].apply(lambda x: "_".join(x).upper(), axis=1)
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price,Sales_Level_Based
0,Girne,Herşey Dahil,High,103.935448,GIRNE_HERŞEY DAHIL_HIGH
1,Girne,Herşey Dahil,Low,90.935944,GIRNE_HERŞEY DAHIL_LOW
2,İzmir,Yarım Pansiyon,High,87.657303,İZMIR_YARIM PANSIYON_HIGH
3,Diğer,Herşey Dahil,Low,87.310882,DIĞER_HERŞEY DAHIL_LOW
4,Diğer,Herşey Dahil,High,83.787273,DIĞER_HERŞEY DAHIL_HIGH


In [16]:
# Müşterileri Price’a göre 4 segmente ayırınız.
# Segmentleri Segment isimlendirmesi ile değişken olarak agg_df’e ekleyiniz.
# Segmentleri betimleyiniz (Segmentlere göre group by yapıp price mean, max, sum’larını alınız).
agg_df["Segment"] = pd.qcut(agg_df["Price"], 4, labels=["D", "C", "B", "A"])
agg_df.groupby(agg_df["Segment"]).agg({"Price": ["mean", "sum", "max"]})

Price                        
              mean         sum         max
Segment                                   
D        33.366288  300.296592   39.479630
C        44.887654  403.988887   54.141797
B        60.274498  542.470478   64.920065
A        82.467833  742.210498  103.935448

3.POTANSİYEL GETİRİ TAHMİNİ

In [17]:
# Yeni gelen müşterileri sınıflandırıp, ne kadar gelir getirebileceklerini tahmin ediniz.Örnek müşteri profilleri tanımlayalım.
# Antalya’da herşey dahil ve yüksek sezonda tatil yapmak isteyen bir kişinin ortalama ne kadar gelir kazandırması beklenir?
new_user = "ANTALYA_HERŞEY DAHIL_HIGH"
print(agg_df.loc[agg_df["Sales_Level_Based"] == new_user, "Segment"].values[0])
print(round(agg_df.loc[agg_df["Sales_Level_Based"] == new_user, "Price"].mean(), 3))

# B Segmentindedir.
# 64.92 Price getirisi öngörülür.

B
64.92


In [18]:
# Girne’de yarım pansiyon bir otele düşük sezonda giden bir tatilci hangi segmentte yer alacaktır?
new_user2 = "GIRNE_YARIM PANSIYON_LOW"
print(agg_df.loc[agg_df["Sales_Level_Based"] == new_user2, "Segment"].values[0])
print(round(agg_df.loc[agg_df["Sales_Level_Based"] == new_user2, "Price"].mean(), 3))

# C Segmentindedir.
# 48.579 Price getirisi öngörülür.

C
48.579


In [19]:
# Müşteri bilgileri girildiğinde segmentini ve tahmini getirisini hesaplayan bir fonksiyon yazalım.
def estimate(dataframe, column, city, concept, season):
    user = f"{city.upper()}_{concept.upper()}_{season.upper()}"
    print(f"SEGMENT: {dataframe.loc[dataframe[column] == user, 'Segment'].values[0]}")
    print(f"AVERAGE PRICE: {round(dataframe.loc[dataframe[column] == user, 'Price'].values[0], 4)}")

estimate(agg_df, "Sales_Level_Based", "Girne", "Yarım Pansiyon", "Low")

SEGMENT: C
AVERAGE PRICE: 48.5789
